In [1]:
import os

In [2]:
%pwd

'c:\\Users\\naush\\kidney-disease-classification\\research'

In [3]:
os.chdir("../")

In [4]:

%pwd

'c:\\Users\\naush\\kidney-disease-classification'

In [7]:
pip install dagshub


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.10.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached urllib3-1.26.19-py2.py3-none-any.whl.metadata (49 kB)
   ---------------------------------------- 0.0/236.6 kB ? eta -:--:--
   --------------------------------------- 236.6/236.6 kB 14.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/71.5 kB ? eta -:--:--
   ---------------------------------------- 71.5/71.5 kB ? eta 0:00:00
   ---------------------------------------- 0.0/238.4 kB ? eta -:--:--
   --------------------------------------- 238.4/238.4 kB 14.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/139.2 kB ? eta -:--:--
   ---------------------------------------- 139.2/139.2 kB ? eta 0:00:00
   ---------------------------------------- 0.0/12.5 MB ? eta -:--:--
   ---- -------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.2.4 requires httpx>=0.25.0, but you have httpx 0.23.3 which is incompatible.


In [8]:

import dagshub
dagshub.init(repo_owner='nausheen', repo_name='kidney-disease-classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\naush\anaconda3\envs\kidney\lib\site-packages\rich\live.py:229: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=8c52e7ac-d8d0-4d42-b4ef-0f435c63591f&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=aae74e6b977707bceacdeee0d76df26c72b724717920b219fe68bf3d8b57d9d4




Accessing as nausheen

Initialized MLflow to track repo "nausheen/kidney-disease-classification"

Repository nausheen/kidney-disease-classification initialized!

In [9]:

import tensorflow as tf

In [10]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [12]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [13]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/entbappy/Kidney-Disease-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [14]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [15]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [17]:

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    #evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-08-06 11:03:58,433: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-06 11:03:58,435: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-06 11:03:58,437: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 8s 817ms/step - loss: 1.5234 - accuracy: 0.8345
[2024-08-06 11:04:06,492: INFO: common: json file saved at: scores.json]
